# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple

La résultat final est dans la dernière cellule. Toutes les cellules précédentes sont là pour aider à la compréhension, en présentant des résultats intermédiaires.


In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%sqlcmd tables

Name
cog_communes
edc_communes
edc_prelevements
edc_resultats
laposte_communes
mapping_categories


In [3]:
%sql select * from cog_communes

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,de_partition,de_ingestion_date,de_dataset_datetime
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None,2024,2025-02-26,20240220
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None,2024,2025-02-26,20240220
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None,2024,2025-02-26,20240220
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None,2024,2025-02-26,20240220
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None,2024,2025-02-26,20240220
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None,2024,2025-02-26,20240220
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None,2024,2025-02-26,20240220
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None,2024,2025-02-26,20240220
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None,2024,2025-02-26,20240220
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None,2024,2025-02-26,20240220


In [4]:
%sql select *  from edc_communes

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date,de_dataset_datetime
01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2020,2025-02-14,20230811-150005
01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2020,2025-02-14,20230811-150005
01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Est,001000850,BDS CHATANIER,2023-01-01,2020,2025-02-14,20230811-150005
01005,AMBERIEUX-EN-DOMBES,Ouest,001000850,BDS CHATANIER,2021-08-01,2020,2025-02-14,20230811-150005
01006,AMBLEON,-,001000235,AMBLEON,2010-09-07,2020,2025-02-14,20230811-150005
01007,AMBRONAY,-,001000003,AMBRONAY,2010-09-07,2020,2025-02-14,20230811-150005
01008,AMBUTRIX,Ambutrix centre,001000254,AMBUTRIX MAIRIE,2010-09-07,2020,2025-02-14,20230811-150005


In [5]:
%sql select *  from edc_prelevements

Running query in 'duckdb:///../../database/data.duckdb'

cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date,de_dataset_datetime
001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100121589,2020-07-08,09h05,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122035,2020-08-13,10h44,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100122549,2020-09-14,12h51,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,None,None,None,00100123096,2020-10-12,08h25,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005
001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100123173,2020-10-12,09h15,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-14,20230811-150005


In [6]:
%%sql
    select *
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL' and valtraduite > 0.5

# on liste l'ensemble des analyses non conformes pour le paramètre CVM

Running query in 'duckdb:///../../database/data.duckdb'

cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
002,00200180193,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,0",µg/L,133,<=0.5 µg/L,None,1.0,75-01-4,00200181303,2020,2025-02-14,20230811-150005
004,00400122403,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,64",µg/L,133,<=0.5 µg/L,None,0.64,75-01-4,00400122467,2020,2025-02-14,20230811-150005
004,00400122776,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,529",µg/L,133,<=0.5 µg/L,None,0.529,75-01-4,00400122840,2020,2025-02-14,20230811-150005
004,00400123465,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,531",µg/L,133,<=0.5 µg/L,None,0.531,75-01-4,00400123529,2020,2025-02-14,20230811-150005
004,00400123960,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"2,00",µg/L,133,<=0.5 µg/L,None,2.0,75-01-4,00400124024,2020,2025-02-14,20230811-150005
004,00400124119,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,40",µg/L,133,<=0.5 µg/L,None,1.4,75-01-4,00400124183,2020,2025-02-14,20230811-150005
005,00500110055,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"0,79",µg/L,133,<=0.5 µg/L,None,0.79,75-01-4,00500117171,2020,2025-02-14,20230811-150005
005,00500112175,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"2,575",µg/L,133,<=0.5 µg/L,None,2.575,75-01-4,00500119367,2020,2025-02-14,20230811-150005
005,00500112309,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,396",µg/L,133,<=0.5 µg/L,None,1.396,75-01-4,00500119512,2020,2025-02-14,20230811-150005
006,00600221382,CLVYL,1753,CHLORURE DE VINYL MONOMÈRE,Chlorure de vinyl monomère,None,N,L,"1,7",µg/L,133,<=0.5 µg/L,None,1.7,75-01-4,00600221398,2020,2025-02-14,20230811-150005


In [7]:
%%sql

with "resultats_cvm" as (
    select
      *,
      (CASE WHEN valtraduite > 0.5 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL'
),
"prelevements_cvm" as (
    select
        "cdreseau",
        "resultats_cvm"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total",
        string_agg(CASE WHEN is_non_conforme = 1 THEN "resultats_cvm"."referenceprel" ELSE null END) as "list_referenceprels_non_conformes",
        string_agg(CASE WHEN is_non_conforme = 1 THEN "valtraduite" ELSE null END) as "list_valtraduite_non_conformes"
    from "resultats_cvm"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm"."de_partition"
    group by "cdreseau", "resultats_cvm"."de_partition"
)
select * from "prelevements_cvm" where "nbr_resultats_non_conformes" > 0

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,de_partition,nbr_resultats_non_conformes,nbr_resultats_total,list_referenceprels_non_conformes,list_valtraduite_non_conformes
005001358,2021,5,15,"00500113232,00500113236,00500113239,00500113835,00500113836","0.65,14.0,0.57,9.1,23.0"
021001615,2021,1,2,02100154905,1.5
023000002,2021,1,3,02300071195,0.69
007001617,2023,11,14,"00700189548,00700190316,00700190913,00700190928,00700191553,00700192134,00700192575,00700194572,00700194644,00700194646,00700194647","0.82,1.1,1.2,0.83,0.82,2.0,1.4,3.6,0.98,1.6,0.64"
015001494,2023,1,2,01500115971,1.1
015002359,2023,2,3,"01500116763,01500116909","0.85,0.83"
015002362,2023,1,2,01500117647,1.4
017000436,2023,2,12,"01700135923,01700138103","0.58,0.51"
018000908,2023,1,6,01800092602,2.0
019000139,2023,3,4,"01900121211,01900121289,01900122845","0.88,1.5,0.88"


In [8]:
%%sql

with "resultats_cvm" as (
    select
      *,
      (CASE WHEN valtraduite > 0.5 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL'
),
"prelevements_cvm" as (
    select
        "cdreseau",
        "resultats_cvm"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total"
    from "resultats_cvm"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm"."de_partition"
    group by "cdreseau", "resultats_cvm"."de_partition"
),
"communes_cvm" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        coalesce(sum("nbr_resultats_non_conformes"), 0) as "nbr_resultats_non_conformes",
        coalesce(sum("nbr_resultats_total"), 0) as "nbr_resultats_total",
        case
            when sum("nbr_resultats_non_conformes") > 0 then 'non conforme'
            when sum("nbr_resultats_total") > 0 then 'conforme'
            else 'non analysé'
        end as "resultat"
    from "edc_communes"
    left join "prelevements_cvm" on
        "prelevements_cvm"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_cvm"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
)
select * from communes_cvm where "nbr_resultats_non_conformes" > 0


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,de_partition,nbr_resultats_non_conformes,nbr_resultats_total,resultat
71101,2020,2,7,non conforme
71121,2020,2,7,non conforme
71175,2020,2,7,non conforme
71364,2020,2,7,non conforme
71396,2020,2,7,non conforme
71537,2020,3,7,non conforme
72153,2020,1,2,non conforme
72364,2020,1,3,non conforme
72369,2020,1,3,non conforme
81009,2020,1,7,non conforme


In [9]:
%%sql

with "resultats_cvm" as (
    select
      *,
      (CASE WHEN valtraduite > 0.5 THEN 1 ELSE 0 END) AS is_non_conforme
    from "edc_resultats"
    where cdparametresiseeaux = 'CLVYL'
),
"prelevements_cvm" as (
    select
        "cdreseau",
        "resultats_cvm"."de_partition",
        SUM(is_non_conforme) as "nbr_resultats_non_conformes",
        count(*) as "nbr_resultats_total"
    from "resultats_cvm"
    left join "edc_prelevements" on
        "edc_prelevements"."referenceprel" = "resultats_cvm"."referenceprel"
        and
        "edc_prelevements"."de_partition" = "resultats_cvm"."de_partition"
    group by "cdreseau", "resultats_cvm"."de_partition"
),
"communes_cvm" as (
    select
        "inseecommune",
        "edc_communes"."de_partition",
        coalesce(sum("nbr_resultats_non_conformes"), 0) as "nbr_resultats_non_conformes",
        coalesce(sum("nbr_resultats_total"), 0) as "nbr_resultats_total",
        case
            when sum("nbr_resultats_non_conformes") > 0 then 'non conforme'
            when sum("nbr_resultats_total") > 0 then 'conforme'
            else 'non analysé'
        end as "resultat"
    from "edc_communes"
    left join "prelevements_cvm" on
        "prelevements_cvm"."cdreseau" = "edc_communes"."cdreseau"
        and
        "prelevements_cvm"."de_partition" = "edc_communes"."de_partition"
    group by "inseecommune", "edc_communes"."de_partition"
),
"annees" as (SELECT unnest(generate_series(2020, 2024)) as "annee")
select
    "cog"."COM" as "commune_code_insee",
    "cog"."LIBELLE" as "commune_nom",
    a."annee",
    coalesce("resultat", 'non analysé') as "resultat"
from "cog_communes" as "cog"
cross join 
    "annees" a
left join "communes_cvm" on
   "cog"."COM" = "communes_cvm"."inseecommune"
   and
   a."annee"::string =  "communes_cvm"."de_partition"

# pour tester une commune avec un prélèvement non conforme, ajouter :
# where "commune_code_insee" = '07194'

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,annee,resultat
01001,L'Abergement-Clémenciat,2020,conforme
01002,L'Abergement-de-Varey,2020,non analysé
01004,Ambérieu-en-Bugey,2020,conforme
01005,Ambérieux-en-Dombes,2020,conforme
01006,Ambléon,2020,non analysé
01007,Ambronay,2020,conforme
01008,Ambutrix,2020,conforme
01009,Andert-et-Condon,2020,conforme
01010,Anglefort,2020,conforme
01011,Apremont,2020,conforme
